# TP3 - 7.4 - SVM - Adrián José Zapater Reig

## 7.4.1 - Clasificación:
- Limpieza de datos
    - Estudio estadístico de room_type
- Clasificación usando SVC y LinearSVC
    - Afinación de SVC usando RBF
        - Encontrar el los mejores valores de gamma y c con GridSearchCV
        - Hacer 2+ busquedas usando numpy.logspace para generar las listas de parámetros. 
    

In [1]:
import os
import pandas as pd

In [2]:
data_path = os.path.join("data","airbnb.csv")
airbnb_raw = pd.read_csv(data_path)

In [3]:
airbnb_raw.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Centro,Justicia,40.424715,-3.698638,Entire home/apt,49,28,35,0.42,1,99
1,Centro,Embajadores,40.413418,-3.706838,Entire home/apt,80,5,18,0.30,1,188
2,Moncloa - Aravaca,Argüelles,40.424920,-3.713446,Entire home/apt,40,2,21,0.25,9,195
3,Moncloa - Aravaca,Casa de Campo,40.431027,-3.724586,Entire home/apt,55,2,3,0.13,9,334
4,Latina,Cármenes,40.403410,-3.740842,Private room,16,2,23,0.76,2,250


### En primer lugar estudiemos room_type

In [4]:
# Tenemos 3 etiquetas a clasificar: ['Entire home/apt' 'Private room' 'Shared room']
print(airbnb_raw['room_type'].unique())

['Entire home/apt' 'Private room' 'Shared room']


In [5]:
# Las etiquetas están repartidas de la siguiente manera:
airbnb_raw.groupby("room_type").size()

room_type
Entire home/apt    7926
Private room       5203
Shared room         192
dtype: int64

## Estudio estadístico:
En esta sección vamos a limpiar el dataset, escalar valores numéricos y transformar los datos categóricos a numéricos.

Como ya sabemos, los modelos SVM son muy sensibles a los valores no escaldos. Para maximizar su rendimiento, escalaremos estos valores con el StandardScaler de scikit learn.

Dado que neighbourhood y neighbourhood_group nos dan la misma información (siendo neighbourhood mas precisa) vamos a descartar neighbourhood_group del dataset.

Vamos a estudiar si hubiesen datos faltantes en alguna columna y sustituirlos por valores neutros (media de la columna para numéricos y eliminar el registro para categóricos puesto que no podemos darle un valor sin afectar a las conclusiones que se puedan sacar.)

In [22]:
airbnb_staged = airbnb_raw.drop("neighbourhood_group", 1, inplace=False)

In [ ]:

# NOTA: PAsar a después del test y train set. Meter aquí limpieza de datos.

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("num_scaler", StandardScaler()),
    ("cat_ordinal", OrdinalEncoder())
    
])


Como podemos ver las etiquetas están desbalanceadas. Shared room tiene muchos registros que las primeras 2.

#### Separar un test set
Usamos stratified test

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit
# Separamos en 2 grupos con 0.2 en el test set.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Separamos el test y train set respetando la proporción de muestras por neighbourhood_group para tener representación 
# de todas las zonas en ambos conjuntos.
for train_id, test_id in split.split(airbnb_raw, airbnb_raw["neighbourhood_group"]):
    train_set_raw, test_set_raw = airbnb_raw.loc[train_id], airbnb_raw.loc[test_id]

In [19]:
train_set_raw["room_type"].count()

10656

In [20]:
test_set_raw["room_type"].count()

2665

### Estudio estad